In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-may-2022/train.csv
/kaggle/input/tabular-playground-series-may-2022/test.csv


In [12]:
# Memory management 
import gc 

#modelling
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

In [2]:
# Load date sets.

train_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')

test_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
test_ids = test_data['id'] 

# target


train_data.head()


,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,...,-2.540739,0.766952,-2.730628,-0.208177,1.363402,ABABDADBAB,67.609153,0,0,0
1,1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,...,2.278315,-0.633658,-1.217077,-3.782194,-0.058316,ACACCADCEB,377.096415,0,0,1
2,2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,...,-1.385775,-0.520558,-0.009121,2.788536,-3.703488,AAAEABCKAD,-195.599702,0,2,1
3,3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,...,0.572594,-1.653213,1.686035,-2.533098,-0.608601,BDBBAACBCB,210.826205,0,0,1
4,4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,...,-3.912929,-1.430366,2.127649,-3.306784,4.371371,BDBCBBCHFE,-217.211798,0,1,1
5,5,1.349785,1.100418,0.562612,-0.607076,1.264728,0.283515,-1.131953,6,2,...,-2.259781,-1.308329,-2.216110,1.016892,0.273792,BDAEDBDEDA,45.890998,0,1,0
6,6,1.904853,-0.174135,0.193908,-0.559832,-0.146990,-0.538386,0.270837,2,5,...,-2.832988,2.432185,-1.827240,1.192390,-0.333300,ACBCAAAHHE,358.058637,1,0,1
7,7,0.889100,0.388168,1.062274,-1.775401,0.852494,0.335741,0.510709,2,3,...,7.956038,2.663890,-0.593400,1.528621,0.972924,BBBDBBEPDB,-113.797366,1,1,0
8,8,0.866221,0.842351,-1.367634,1.237506,0.682962,0.845666,0.173852,3,2,...,-1.241121,-1.245038,1.696402,0.354545,-0.541267,ACAEBADDAA,-52.223857,0,0,0
9,9,0.829142,-0.082872,-0.206667,-0.032502,0.366289,0.277981,-1.113467,2,0,...,-3.117106,-3.609328,2.677487,-1.772600,-3.053609,BABCBBBABD,-42.976312,0,0,1


In [3]:
#check for unique values
for col in test_data:
    print(col)
    print(test_data[col].unique())

id
[ 900000  900001  900002 ... 1599997 1599998 1599999]
f_00
[ 0.44251689 -0.60559756  0.30398973 ... -0.33170369 -2.03107297
 -0.08590636]
f_01
[ 1.74380235e-01 -3.05714509e-01  2.44511024e+00 ... -3.28844814e-01
 -1.23839802e+00 -2.12377493e-03]
f_02
[-0.99981593  0.62766725  0.24651454 ... -1.18550262  0.96469876
  2.22737467]
f_03
[ 0.76274101 -0.57889756  0.81824834 ...  1.02212795 -1.04595013
  0.21714491]
f_04
[ 0.18677843 -1.75093096  0.35973054 ... -0.48309949  0.90606413
  3.17915307]
f_05
[-1.0747749   1.35555045 -1.33184544 ... -0.10714629  0.63430066
 -1.66018827]
f_06
[ 0.50188752 -0.19091089  1.35862229 ... -0.96828118 -0.70747429
  0.89198907]
f_07
[ 6  1  3  0  2  5  8  4  7 12  9 10 11 13 15 16]
f_08
[ 6  3  0  2  1  4  5  8 11  7 10  9 12 13]
f_09
[ 0  4  2  3  9  5  1  6  7  8 10 12 11 13 16 14]
f_10
[ 0  1  2  4  3  6  5  7  8  9 10 12 11 14 15 13]
f_11
[ 0  2  1  3  5  4  7  6  8  9 11 10 12 13 14]
f_12
[ 5  1  3  4  6  2  7  0  9  8 10 11 12 13 14 15]
f_13
[ 1  

In [4]:
#look for any NAN values and types

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900000 entries, 0 to 899999
Data columns (total 33 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      900000 non-null  int64  
 1   f_00    900000 non-null  float64
 2   f_01    900000 non-null  float64
 3   f_02    900000 non-null  float64
 4   f_03    900000 non-null  float64
 5   f_04    900000 non-null  float64
 6   f_05    900000 non-null  float64
 7   f_06    900000 non-null  float64
 8   f_07    900000 non-null  int64  
 9   f_08    900000 non-null  int64  
 10  f_09    900000 non-null  int64  
 11  f_10    900000 non-null  int64  
 12  f_11    900000 non-null  int64  
 13  f_12    900000 non-null  int64  
 14  f_13    900000 non-null  int64  
 15  f_14    900000 non-null  int64  
 16  f_15    900000 non-null  int64  
 17  f_16    900000 non-null  int64  
 18  f_17    900000 non-null  int64  
 19  f_18    900000 non-null  int64  
 20  f_19    900000 non-null  float64
 21  f_20    90

In [5]:
# check for dups Row

train_data[train_data.duplicated()]

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target


In [6]:
# column duplicates

train_data.apply(lambda x : x.duplicated().sum())

id             0
f_00           0
f_01           0
f_02           0
f_03           0
f_04           0
f_05           0
f_06           0
f_07      899984
f_08      899984
f_09      899985
f_10      899985
f_11      899986
f_12      899984
f_13      899987
f_14      899986
f_15      899985
f_16      899984
f_17      899986
f_18      899986
f_19           0
f_20           0
f_21           0
f_22           0
f_23           0
f_24           0
f_25           0
f_26           0
f_27      158646
f_28           0
f_29      899998
f_30      899997
target    899998
dtype: int64

📝 Notes:

* f_27 has text with many Dups
* No nan Values. 

* f_29 and f_30 look like classes.

* f_29 [0 1]
* f_30 [0 1 2]

* f_27
text split or other features to extract.

In [16]:
# Function to Clean up the data


#reload data

train_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')

test_data = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')



def clean(data):
    
    data = data.drop(['id'], axis = 1)
    
    data = data.drop(['f_27'], axis = 1) # add features to this data point

        
        
    return data
    
    
    
        
train_data = clean(train_data)
test_data = clean(test_data)

train_data.head()
    
       
y = train_data["target"]
X = train_data.drop("target", axis=1)

    
    
    
    
X_train.head()


,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,...,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_28,f_29,f_30
0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,1,...,-0.919717,3.058541,-2.540739,0.766952,-2.730628,-0.208177,1.363402,67.609153,0,0
1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,4,...,-1.075434,2.179050,2.278315,-0.633658,-1.217077,-3.782194,-0.058316,377.096415,0,0
2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,2,...,-3.485342,-0.784235,-1.385775,-0.520558,-0.009121,2.788536,-3.703488,-195.599702,0,2
3,-0.118172,-0.587835,-0.804638,2.086822,0.371005,-0.128831,-0.282575,3,2,1,...,-2.100177,-2.343819,0.572594,-1.653213,1.686035,-2.533098,-0.608601,210.826205,0,0
4,1.148481,-0.176567,-0.664871,-1.101343,0.467875,0.500117,0.407515,3,3,0,...,0.605033,1.133665,-3.912929,-1.430366,2.127649,-3.306784,4.371371,-217.211798,0,1


In [18]:
#train test split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)





In [21]:
EPOCHS = 5000

In [22]:
model = lgb.LGBMClassifier(n_jobs = -1, n_estimators = EPOCHS)
model.fit(X_train,y_train, eval_set=[(X_test,y_test)], callbacks = [lgb.early_stopping(30)],eval_metric="auc")




Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[2952]	valid_0's auc: 0.936291	valid_0's binary_logloss: 0.324821


LGBMClassifier(n_estimators=5000)

In [23]:
val_preds = model.predict_proba(X_test)
y_preds = model.predict_proba(X_train)

print("Intrinsic AUC:", roc_auc_score(y_train, y_preds[:,1]))
print("Validation AUC:", roc_auc_score(y_test, val_preds[:, 1] ))


Intrinsic AUC: 0.9657800996884387
Validation AUC: 0.9362911824618043


In [17]:
test_data.head()

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,...,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_28,f_29,f_30
0,0.442517,0.174380,-0.999816,0.762741,0.186778,-1.074775,0.501888,6,6,0,...,2.749347,-1.006400,-1.193879,-2.435736,-2.427430,-1.966887,5.734205,99.478419,0,0
1,-0.605598,-0.305715,0.627667,-0.578898,-1.750931,1.355550,-0.190911,1,3,4,...,1.080762,2.382405,0.149442,1.883322,-2.848714,-0.725155,3.194219,-65.993825,1,0
2,0.303990,2.445110,0.246515,0.818248,0.359731,-1.331845,1.358622,3,3,4,...,-1.406712,-7.026098,1.312277,-5.157192,1.714005,0.585032,0.066898,-87.405622,0,1
3,0.154053,0.260126,-1.367092,-0.093175,-1.111034,-0.948481,1.119220,0,0,4,...,3.011085,-0.594532,-3.939475,1.754570,-2.364007,-1.003320,3.893099,-281.293460,0,0
4,-1.651904,-0.424266,-0.667356,-0.322124,-0.089462,0.181705,1.784983,2,2,2,...,0.100594,0.084906,-0.985736,-0.130467,-3.557893,1.210687,1.861884,25.629415,0,2


In [42]:
#Submission 

sub_preds = model.predict_proba(test_data)[:, 1]

sub_preds.size

700000

2

In [39]:
test_ids.size

700000

In [44]:
#m aking the submissions data frame

df = pd.DataFrame({'id':test_ids.values,
                 "target":sub_preds})

df.head()

,id,target
0,900000,0.965565
1,900001,0.914582
2,900002,0.000307
3,900003,0.056603
4,900004,0.744227


In [45]:
df.to_csv("MayTabPlaySub1.csv", index = False)